In [ ]:
'''
Código para verificar se compensa realizar um parcelamento ou pagar à vista com
base no valor da compra.
'''

def obter_opcao(mensagem, opcoes):
    """
    Função genérica para validar e obter uma opção de entrada do usuário.
    :param mensagem: Mensagem exibida ao usuário.
    :param opcoes: Lista de opções válidas.
    :return: Opção escolhida pelo usuário.
    """
    while True:
        try:
            opcao = int(input(mensagem))
            if opcao in opcoes:
                return opcao
            else:
                print("Opção inválida! Tente novamente.")
        except ValueError:
            print("Entrada inválida! Digite um número.")

def obter_valor(mensagem):
    """
    Função para validar e obter um valor decimal do usuário.
    :param mensagem: Mensagem exibida ao usuário.
    :return: Valor decimal informado pelo usuário.
    """
    while True:
        try:
            valor = float(input(mensagem).replace(",", "."))
            if valor > 0:
                return valor
            else:
                print("O valor deve ser positivo! Tente novamente.")
        except ValueError:
            print("Entrada inválida! Digite um valor numérico.")

def calcular_taxa_mensal():
    """
    Calcula a taxa de rendimento mensal com base na escolha do usuário.
    :return: Taxa de rendimento mensal em formato decimal.
    """
    opcao = obter_opcao('Escolha a taxa de rendimento: \n[1] Mensal\n[2] Anual\nOpção: ', [1, 2])
    if opcao == 1:
        return obter_valor('Digite o valor da taxa mensal de rendimento (%): ') / 100
    else:
        taxa_anual = obter_valor('Digite o valor da taxa anual de rendimento (%): ') / 100
        return (1 + taxa_anual)**(1 / 12) - 1

def ajustar_taxa_para_ir(taxa, qnt_parcelas):
    """
    Ajusta a taxa mensal de rendimento com base no imposto de renda, caso o usuário opte por considerar.
    :param taxa: Taxa de rendimento mensal original.
    :param qnt_parcelas: Quantidade de parcelas.
    :return: Taxa ajustada considerando ou não o IR.
    """
    considerar_ir = input('Deseja considerar o imposto de renda? [S/N]: ').strip().upper()
    while considerar_ir not in ['S', 'N']:
        print('Resposta inválida! Tente novamente.')
        considerar_ir = input('Deseja considerar o imposto de renda? [S/N]: ').strip().upper()

    if considerar_ir == 'S':
        if qnt_parcelas <= 6:
            return taxa * 0.775  # 22.5% de IR
        elif qnt_parcelas <= 12:
            return taxa * 0.80  # 20% de IR
        elif qnt_parcelas <= 24:
            return taxa * 0.825  # 17.5% de IR
        else:
            return taxa * 0.85  # 15% de IR
    else:
        print("A taxa de rendimento será utilizada sem ajustes de IR.")
        return taxa

def calcular_valor_parcelado(valor_inicial, qnt_parcelas):
    """
    Calcula o valor total e de cada parcela com possíveis juros.
    :param valor_inicial: Valor inicial da compra.
    :param qnt_parcelas: Quantidade de parcelas.
    :return: Valor total ajustado e valor de cada parcela.
    """
    tem_juros = input('Há juros em pagar parcelado? [S/N]: ').strip().upper()
    while tem_juros not in ['S', 'N']:
        print('Resposta inválida! Tente novamente.')
        tem_juros = input('Há juros em pagar parcelado? [S/N]: ').strip().upper()

    if tem_juros == 'S':
        tipo_juros = obter_opcao('Escolha o tipo de juros: \n[1] Simples\n[2] Compostos\nOpção: ', [1, 2])
        if tipo_juros == 1:
            juros = obter_valor('Qual a porcentagem de juros em pagar parcelado? ')
            valor_total = valor_inicial * (1 + (juros / 100))
        else:
            periodo_juros = obter_opcao('Escolha o período de juros: \n[1] Diário\n[2] Mensal\n[3] Anual\nOpção: ', [1, 2, 3])
            if periodo_juros == 1:
                juros_diarios = obter_valor('Qual a porcentagem de juros ao dia (a.d.)? ') / 100
                juros_mensais = (1 + juros_diarios)**30 - 1
            elif periodo_juros == 2:
                juros_mensais = obter_valor('Qual a porcentagem de juros ao mês (a.m.)? ') / 100
            else:
                juros_anuais = obter_valor('Qual a porcentagem de juros ao ano (a.a.)? ') / 100
                juros_mensais = (1 + juros_anuais)**(1 / 12) - 1

            valor_total = valor_inicial * (1 + juros_mensais)**qnt_parcelas
    else:
        valor_total = valor_inicial

    return valor_total, valor_total / qnt_parcelas

def simular_rendimento(qnt_parcelas, valor_parcela, taxa_mensal):
    """
    Simula os rendimentos acumulados com as parcelas mensais.
    :param qnt_parcelas: Quantidade de parcelas.
    :param valor_parcela: Valor de cada parcela.
    :param taxa_mensal: Taxa de rendimento mensal ajustada.
    :return: Rendimentos acumulados.
    """
    ganho = 0
    divida = valor_parcela * qnt_parcelas
    print(f'\n{"Parcela":^10}{"Dívida (R$)":>20}{"Rendimento (R$)":>20}')
    print('-' * 50)
    for i in range(1, qnt_parcelas + 1):
        rendimento = divida * taxa_mensal
        ganho += rendimento
        divida -= valor_parcela
        print(f'{i:^10}{divida:>20.2f}{rendimento:>20.2f}')
    return ganho

def main():
    """
    Função principal para executar a comparação entre pagamento à vista e parcelado.
    """
    print('Bem vindo ao TaxCalcBR: Simulador de Pagamentos')
    opcao = obter_opcao('Escolha a comparação desejada: \n[1] Rendimento x Parcelas\n[2] À vista x Parcelas\nOpção: ', [1, 2])

    if opcao == 1:
        taxa_mensal = calcular_taxa_mensal()
        valor_inicial = obter_valor('Digite o valor total da compra: R$')
        qnt_parcelas = int(obter_valor('Digite o número de parcelas: '))
        taxa_mensal = ajustar_taxa_para_ir(taxa_mensal, qnt_parcelas)
        print(f'A taxa de rendimento ajustada é: {taxa_mensal:.4f}')

        desconto_vista = input('Há desconto em pagar à vista? [S/N]: ').strip().upper()
        while desconto_vista not in ['S', 'N']:
            print('Resposta inválida! Tente novamente.')
            desconto_vista = input('Há desconto em pagar à vista? [S/N]: ').strip().upper()

        if desconto_vista == 'S':
            desconto = obter_valor('Qual a porcentagem de desconto em pagar à vista? ')
            valor_vista = valor_inicial * (1 - desconto / 100)
        else:
            valor_vista = valor_inicial

        valor_total_parcelado, valor_parcela = calcular_valor_parcelado(valor_inicial, qnt_parcelas)
        rendimentos = simular_rendimento(qnt_parcelas, valor_parcela, taxa_mensal)

        valor_efetivo_parcelado = valor_total_parcelado - rendimentos

        print(f'\nValor pagando à vista: R${valor_vista:.2f}')
        print(f'Valor pagando parcelado: R${valor_total_parcelado:.2f}')
        print(f'Rendimentos acumulados com o parcelamento: R${rendimentos:.2f}')
        print(f'Valor pago parcelando (descontados os rendimentos): R${valor_efetivo_parcelado:.2f}')

        if valor_vista < valor_efetivo_parcelado:
            print('\nCompensa pagar à vista.')
        else:
            print('\nCompensa parcelar.')
    else:
        valor_vista = obter_valor('Digite o valor da compra à vista: R$')
        qnt_parcelas = int(obter_valor('Digite o número de parcelas: '))
        valor_parcela = obter_valor('Digite o valor de cada parcela: R$')
        valor_total_parcelado = qnt_parcelas * valor_parcela
        taxa_mensal = calcular_taxa_mensal()
        taxa_mensal = ajustar_taxa_para_ir(taxa_mensal, qnt_parcelas)

        rendimentos = simular_rendimento(qnt_parcelas, valor_parcela, taxa_mensal)
        valor_efetivo_parcelado = valor_total_parcelado - rendimentos

        print(f'\nValor pagando à vista: R${valor_vista:.2f}')
        print(f'Valor pagando parcelado: R${valor_total_parcelado:.2f}')
        print(f'Rendimentos acumulados com o parcelamento: R${rendimentos:.2f}')
        print(f'Valor pago parcelando (descontados os rendimentos): R${valor_efetivo_parcelado:.2f}')

        if valor_vista < valor_efetivo_parcelado:
            print('\nCompensa pagar à vista.')
        else:
            print('\nCompensa parcelar.')

if __name__ == "__main__":
    main()